<a href="https://colab.research.google.com/github/hyrahmaaa/Calculator-Model-Premi/blob/main/Kalkulator_Model_Premi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kalkulator Model Premi**

## Hitung Nilai Fungsi Massa Peluang W_t

In [3]:
import math
import pandas as pd
# import google.colab.files # Tidak perlu diimpor di sini, akan diimpor saat dibutuhkan

def calculate_pr_Wt_k_for_single_lambda(n, q, lambda_val, t_val):
    """
    Menghitung probabilitas Pr{Wt = k} untuk satu set parameter (n, q, lambda, t).

    Args:
        n (int): Jumlah total percobaan (max nilai k).
        q (float): Probabilitas "kegagalan" (0 <= q <= 1).
        lambda_val (float): Nilai lambda (non-negatif).
        t_val (float): Nilai waktu t (non-negatif).

    Returns:
        list: Sebuah list berisi probabilitas Pr{Wt = k} untuk k = 0 hingga n.
              Mengembalikan list probabilitas nol jika ada error dalam perhitungan e^(-lambda*t).
    """
    p = 1 - q # Probabilitas "sukses"

    probabilities = []

    # Hitung e^(-lambda * t)
    try:
        exp_lambda_t = math.exp(-lambda_val * t_val)
    except OverflowError:
        print(f"Peringatan: Nilai lambda * t ({lambda_val * t_val}) terlalu besar, menyebabkan OverflowError untuk e^(-lambda * t). Mengembalikan probabilitas 0.")
        return [0.0] * (n + 1)
    except Exception as e:
        print(f"Error saat menghitung e^(-lambda * t) untuk lambda={lambda_val}, t={t_val}: {e}")
        return [0.0] * (n + 1)

    for k in range(n + 1):
        if k < n:
            # Formula untuk k = 0, 1, 2, ..., (n-1)
            binomial_coefficient = math.comb(n, k)
            prob_k = binomial_coefficient * (q**k) * (p**(n - k)) * exp_lambda_t
        else: # k == n
            # Formula untuk k = n
            prob_k = (q**n * exp_lambda_t) + (1 - exp_lambda_t)

        probabilities.append(prob_k)

    return probabilities

def generate_probability_table(n, q, t, lambda_values_dict):
    """
    Menghasilkan tabel probabilitas Pr{Wt = k} untuk berbagai nilai lambda.

    Args:
        n (int): Jumlah total percobaan (max nilai k).
        q (float): Probabilitas "kegagalan" (0 <= q <= 1).
        t (float): Nilai waktu t (non-negatif).
        lambda_values_dict (dict): Dictionary dengan key adalah string dari nilai lambda
                                   dan value adalah nilai lambda float yang sesuai.

    Returns:
        pandas.DataFrame: Tabel probabilitas dengan kolom 'k' dan kolom untuk setiap nilai lambda.
    """
    # Basic input validation (more robust validation can be added)
    if not (isinstance(n, int) and n >= 0):
        print("Error: n harus bilangan bulat non-negatif.")
        return pd.DataFrame()
    if not (0 <= q <= 1):
        print("Error: q harus antara 0 dan 1.")
        return pd.DataFrame()
    if t < 0:
        print("Error: t harus non-negatif.")
        return pd.DataFrame()

    # Inisialisasi DataFrame dengan kolom 'k'
    df = pd.DataFrame({'k': list(range(n + 1))})

    # Iterasi untuk setiap nilai lambda yang diberikan
    for lambda_str_name, lambda_val in lambda_values_dict.items():
        if lambda_val < 0:
            print(f"Peringatan: Nilai lambda '{lambda_str_name}' adalah negatif. Kolom ini akan diisi nol.")
            df[lambda_str_name] = [0.0] * (n + 1)
            continue

        probs_for_lambda = calculate_pr_Wt_k_for_single_lambda(n, q, lambda_val, t)
        df[lambda_str_name] = probs_for_lambda

    return df

# --- Bagian Input Pengguna dan Eksekusi ---
if __name__ == "__main__":
    print("--- Generator Tabel Probabilitas Pr{Wt = k} ---")
    print("Masukkan parameter yang diperlukan:")

    try:
        # Input untuk n
        n_input = int(input("Masukkan nilai n (jumlah maksimum k, bilangan bulat non-negatif): "))
        if n_input < 0:
            raise ValueError("n harus non-negatif.")

        # Input untuk q
        q_input = float(input("Masukkan nilai q (probabilitas kegagalan, antara 0 dan 1): "))
        if not (0 <= q_input <= 1):
            raise ValueError("q harus antara 0 dan 1.")

        # Input untuk t
        t_input = float(input("Masukkan nilai t (waktu, non-negatif): "))
        if t_input < 0:
            raise ValueError("t harus non-negatif.")

        # Input untuk nilai-nilai lambda (sekarang langsung nilainya yang jadi nama kolom)
        print("\nMasukkan nilai-nilai lambda yang ingin dijadikan kolom (ketik 'selesai' untuk mengakhiri):")
        lambda_data = {}
        while True:
            lambda_input_str = input("Masukkan nilai lambda (ketik 'selesai' untuk mengakhiri): ").strip()
            if lambda_input_str.lower() == 'selesai' or not lambda_input_str:
                break

            try:
                lambda_val = float(lambda_input_str)
                # Gunakan string dari nilai lambda sebagai key/nama kolom
                lambda_data[lambda_input_str] = lambda_val
            except ValueError:
                print("Input lambda tidak valid. Harap masukkan angka.")

        if not lambda_data:
            print("Tidak ada nilai lambda yang dimasukkan. Tidak ada tabel yang akan dibuat.")
        else:
            # Panggil fungsi untuk menghasilkan tabel
            final_table = generate_probability_table(n_input, q_input, t_input, lambda_data)

            if not final_table.empty:
                print("\n--- Tabel Probabilitas Pr{Wt = k} ---")

                # Mengatur format tampilan Pandas untuk angka float di konsol
                pd.set_option('display.float_format', '{:.6g}'.format)

                # Tampilkan tabel di konsol
                print(final_table.to_string(index=False))

                print("\n--- Cek Total Probabilitas (Per Kolom) ---")
                print(final_table.drop(columns=['k']).sum().to_string(float_format='{:.6g}'.format))

                # --- Tambahan untuk mengunduh tabel ---
                print("\nApakah Anda ingin mengunduh tabel ini? (ya/tidak)")
                download_choice = input().strip().lower()

                if download_choice == 'ya':
                    file_name = "tabel_probabilitas.csv"
                    try:
                        # Menyimpan DataFrame ke CSV
                        # float_format='%.6g' memastikan format angka yang sama seperti tampilan
                        final_table.to_csv(file_name, index=False, float_format='%.6g')
                        print(f"Tabel berhasil disimpan sebagai '{file_name}' di sesi Colab.")

                        # Mengunduh file menggunakan google.colab.files
                        from google.colab import files
                        files.download(file_name)
                        print("Unduhan akan dimulai di browser Anda.")
                    except ImportError:
                        print("Modul 'google.colab.files' tidak ditemukan. Mungkin Anda tidak menjalankan ini di Google Colab.")
                    except Exception as e:
                        print(f"Gagal mengunduh tabel: {e}")
                else:
                    print("Tabel tidak diunduh.")

    except ValueError as ve:
        print(f"Input Error: {ve}")
    except Exception as e:
        print(f"Terjadi kesalahan tak terduga: {e}")

--- Generator Tabel Probabilitas Pr{Wt = k} ---
Masukkan parameter yang diperlukan:
Masukkan nilai n (jumlah maksimum k, bilangan bulat non-negatif): 10
Masukkan nilai q (probabilitas kegagalan, antara 0 dan 1): 0.016992
Masukkan nilai t (waktu, non-negatif): 1

Masukkan nilai-nilai lambda yang ingin dijadikan kolom (ketik 'selesai' untuk mengakhiri):
Masukkan nilai lambda (ketik 'selesai' untuk mengakhiri): 0.000696
Masukkan nilai lambda (ketik 'selesai' untuk mengakhiri): 0.000325
Masukkan nilai lambda (ketik 'selesai' untuk mengakhiri): 0.000128
Masukkan nilai lambda (ketik 'selesai' untuk mengakhiri): 0.000173
Masukkan nilai lambda (ketik 'selesai' untuk mengakhiri): selesai

--- Tabel Probabilitas Pr{Wt = k} ---
 k    0.000696    0.000325    0.000128    0.000173
 0    0.841915    0.842227    0.842393    0.842355
 1    0.145531    0.145585    0.145614    0.145607
 2   0.0113202   0.0113244   0.0113267   0.0113262
 3 0.000521809 0.000522003 0.000522106 0.000522082
 4 1.57847e-05 1.5

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Unduhan akan dimulai di browser Anda.
